In [81]:
import numpy as np
N, T, dX, dU,dV = 10, 100,26, 7, 7

l = np.zeros(T)
lu = np.zeros((T, dU))
lv = np.zeros((T, dV))
lx = np.zeros((T, dX))
luu = np.zeros((T, dU, dU))
lvv = np.zeros((T, dU, dV))
lxx = np.zeros((T, dX, dX))
lux = np.zeros((T, dU, dX))
lvx = np.zeros((T, dV, dX))

# Compute cost.
cs = np.zeros((N, T))  # see algorithm_utils.py
cc = np.zeros((N, T))   # Cost estimate constant term.
cv = np.zeros((N, T, dX+dU+dV))    # Cost estimate vector term.
Cm = np.zeros((N, T, dX+dU+dV, dX+dU+dV)) # Cost estimate matrix term.

term1 = np.c_[lxx, np.transpose(lux, [0, 2, 1]), np.transpose(lvx, [0, 2, 1])]
term2 = np.c_[lux, luu, lvv]
cv_temp = np.c_[lx, lu,  lv]
term3 = np.c_[lvx, lvv]
Cm_ord = np.concatenate(
                (np.c_[lxx, np.transpose(lux, [0, 2, 1])], np.c_[lux, luu]),
                axis=1
            )
Cm_temp = np.concatenate(
            (np.c_[lxx, np.transpose(lux, [0, 2, 1]), np.transpose(lvx, [0, 2, 1])], np.c_[lux, luu, lvv]),
            axis=1
        )   
Cm2 = np.concatenate((Cm_temp, np.c_[lvx, lvv, luu],), axis=1)
# # print('term1: {}, Cm: {}, term2: {}, term3: {}'.format(term1.shape, Cm.shape, term2.shape, term3.shape))
print('Cm : {} temp'.format(Cm_temp.shape), np.c_[lxx, np.transpose(lux, [0, 2, 1])].shape)
print(np.c_[lxx, np.transpose(lux, [0, 2, 1]), np.transpose(lvx, [0, 2, 1])].shape)
print(np.c_[lux, luu, lvv].shape)
print(np.c_[lvx, lvv, luu].shape)
print('Cm temp: ', Cm_temp.shape)
print('Cm2 shape', Cm2.shape)
for n in range(N):
    cc[n, :] = l
    cs[n, :] = l
    cv[n,:,:] = np.c_[lx, lu,  lv]
    Cm_temp = np.concatenate(
            (np.c_[lxx, np.transpose(lux, [0, 2, 1]), np.transpose(lvx, [0, 2, 1])], np.c_[lux, luu, lvv]),
            axis=1
        )     
    Cm[n, :, :, :] = np.concatenate((Cm_temp, np.c_[lvx, lvv, luu],), axis=1)

print('cs: {}, cv: {}, Cm: {}'.format(cs.shape, cv.shape, Cm.shape))

('Cm : (100, 33, 40) temp', (100, 26, 33))
(100, 26, 40)
(100, 7, 40)
(100, 7, 40)
('Cm temp: ', (100, 33, 40))
('Cm2 shape', (100, 40, 40))
cs: (10, 100), cv: (10, 100, 40), Cm: (10, 100, 40, 40)


In [71]:
import numpy as np

dX, dU, dV = 26, 7, 7

Ltt = np.diag(np.hstack([
     np.ones(dU),
       np.ones(dU),
    np.zeros(dX - dU*2), np.ones(dU)
]))

Ltt2 = np.diag(np.hstack([
     np.ones(dU),
     np.ones(dU),
    np.zeros(dX - dU*2), np.ones(dU),
     np.ones(dV)])
              )
print('Ltt: ', Ltt.shape)
print('Ltt2: ', Ltt2.shape)
# x0 = np.concatenate([np.array([0.1, 0.1, -1.54, -1.7, 1.54, -0.2, 0]),
#                      np.zeros(7)])
# print(x0.shape, np.r_[x0, np.zeros(dU)].shape)
# lt = -Ltt.dot(np.r_[x0, np.zeros(dU)])

('Ltt: ', (33, 33))
('Ltt2: ', (40, 40))


In [72]:
import numpy as np
dX, dU, dV = 26, 7, 7
dt = 0.1
gains = 1.0/np.array([3.09, 1.08, 0.393, 0.674, 0.111, 0.152, 0.098])
Fd = np.vstack([
        np.hstack([
            np.eye(dU), dt * np.eye(dU), np.zeros((dU, dX - dU*2)),
            dt ** 2 * np.diag(gains)
        ]),
        np.hstack([
            np.zeros((dU, dU)), np.eye(dU), np.zeros((dU, dX - dU*2)),
            dt * np.diag(gains)
        ]),
        np.zeros((dX - dU*2, dX+dU))
    ])
acc = np.zeros(7)
fc = np.hstack([acc * dt ** 2, acc * dt, np.zeros((dX - dU*2))])

Fdr = np.vstack([
        np.hstack([
            np.eye(dU), dt * np.eye(dU), np.zeros((dU, dX - dU*2)),
            dt ** 2 * np.diag(gains),
            np.eye(dV), #dt * np.eye(dV), np.zeros((dV, dX - dV*2)),
           # dt ** 2 * np.diag(gains),
        ]),
        np.hstack([
            np.zeros((dU, dU)), np.eye(dU), np.zeros((dU, dX - dU*2)),
            dt * np.diag(gains),
            np.zeros((dV, dV)), #np.eye(dV), np.zeros((dV, dX - dV*2)),
            #dt ** 2 * np.diag(gains),
        ]),
       np.zeros((dX - dU*2, dX+dU+dV))
    ])
fcr = np.hstack([acc * dt ** 2, acc * dt, np.zeros((dX - dU))])
print('fc: {}, Fd: {}, Fdr: {}, fcr: {}'.format(fc.shape, Fd.shape, Fdr.shape, fcr.shape))

fc: (26,), Fd: (26, 33), Fdr: (26, 40), fcr: (33,)


In [82]:
T, dU = 100, 7
O = np.zeros((0, T, dU))
print(O.shape)

(0, 100, 7)


In [69]:
import scipy as sp
help(sp.linalg.solve_triangular)

AttributeError: 'module' object has no attribute 'linalg'